In [1]:
%load_ext autoreload
%autoreload 2

import os
import time

import sys
sys.path.append(os.environ['GORDON_REPO_DIR'] + '/pipeline_scripts')

import utilities2014
reload(utilities2014)
from utilities2014 import *

sys.path.append('/home/yuncong/project/opencv-2.4.9/release/lib/python2.7/site-packages')
import cv2

from joblib import Parallel, delayed

from skimage.color import gray2rgb
from skimage.util import img_as_float, pad
from skimage.morphology import disk
from skimage.filters.rank import gradient

import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
section_ind = 3
dm = DataManager(generate_hierarchy=False, stack='RS141', resol='x5', section=section_ind)
dm._load_image()

texton_hists = dm.load_pipeline_result('texHist', 'npy')
segmentation = dm.load_pipeline_result('segmentation', 'npy')
n_superpixels = len(np.unique(segmentation)) - 1
textonmap = dm.load_pipeline_result('texMap', 'npy')
n_texton = len(np.unique(textonmap)) - 1

texture_map = dm.load_pipeline_result('textureMap', 'npy')
Gmax = dm.load_pipeline_result('Gmax', 'npy')
Gmax_viz = plt.cm.jet(Gmax/np.nanmax(Gmax))

thetas = np.linspace(-np.pi/4, np.pi/4, 9)
n_theta = len(thetas)
Rs = [np.array([[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]]) for theta in thetas]

In [13]:
lm_ind = 3

lm = np.load('/home/yuncong/csd395/sample_landmark_%d.npy'%lm_ind)

lm_int_texture = lm[:, 2:2+n_texton]
lm_ext_texture = lm[:, 2+n_texton:-2]
lm_vector = lm[:, -2:]

lm_pts = lm[:, [1,0]]
lm_pts_centered = lm_pts - lm_pts.mean(axis=0)

template_points_rotated_versions = []
for R in Rs:
    lm_pts_rotated = np.dot(R, lm_pts_centered.T).astype(np.int).T
    lm_pts_rotated_shifted = lm_pts_rotated - lm_pts_rotated.min(axis=0)
    template_points_rotated_versions.append(lm_pts_rotated_shifted)

template_shape_rotated_versions = [pts.max(axis=0) + [1,1] for pts in template_points_rotated_versions]

In [31]:
locs = np.load('/home/yuncong/csd395/top3LocsLm%dSect%d.npy'%(lm_ind, section_ind))
shifted_ymean_best, shifted_xmean_best, v, orientation = locs[0]
template_points = template_points_rotated_versions[int(orientation)]
shifted_ys_best = (shifted_ymean_best - template_points[:,0].mean(axis=0) + template_points[:,0]).astype(np.int)
shifted_xs_best = (shifted_xmean_best - template_points[:,1].mean(axis=0) + template_points[:,1]).astype(np.int)

In [87]:
def snap(pts, contrast_map, snap_explore_radius = 50, iterations = 1, texture_sample_radius = 5):
    
    pts = pts.copy()
    n = len(pts)
    rs = np.r_[np.arange(-snap_explore_radius, 0), np.arange(1, snap_explore_radius+1)]
    int_rs = np.arange(-texture_sample_radius, 0)
    ext_rs = np.arange(1, texture_sample_radius+1)
    
    for _ in range(iterations):
    
        D = cdist(pts, pts)
        pts_sorted = D.argsort(axis=1)[:,:20]

        gradient_dirs = np.empty((pts.shape[0], 2))
        for i, neighborhood in enumerate(pts_sorted):
            X = pts[neighborhood]
            c = X.mean(axis=0)
            Xc = X - c
            U,S,V = np.linalg.svd(np.dot(Xc.T, Xc))
            tangent_dir = U[:,0]
            gradient_dirs[i] = np.array([tangent_dir[1], -tangent_dir[0]])
            if np.dot(gradient_dirs[i], lm_vector[i]) < 0:
                gradient_dirs[i] = -gradient_dirs[i]
    
        sample_ys = pts[:,0][:,None] + np.outer(gradient_dirs[:,0], rs).astype(np.int)
        sample_xs = pts[:,1][:,None] + np.outer(gradient_dirs[:,1], rs).astype(np.int)

        
        int_texture_sample_ys = np.array([np.add.outer(sample_ys[i], np.outer(gradient_dirs[:,0], int_rs)[i].astype(np.int)) 
                                          for i in range(n)])
        int_texture_sample_xs = np.array([np.add.outer(sample_xs[i], np.outer(gradient_dirs[:,1], int_rs)[i].astype(np.int)) 
                                          for i in range(n)])
        avg_int_texture_samples = texture_map[int_texture_sample_ys, int_texture_sample_xs].mean(axis=2)
        int_chi2s = np.array([chi2s(lm_int_texture, avg_int_texture_samples[:,i]) for i in range(len(rs))]).T
        
        ext_texture_sample_ys = np.array([np.add.outer(sample_ys[i], np.outer(gradient_dirs[:,0], ext_rs)[i].astype(np.int)) for i in range(n)])
        ext_texture_sample_xs = np.array([np.add.outer(sample_xs[i], np.outer(gradient_dirs[:,1], ext_rs)[i].astype(np.int)) for i in range(n)])
        avg_ext_texture_samples = texture_map[ext_texture_sample_ys, ext_texture_sample_xs].mean(axis=2)
        ext_chi2s = np.array([chi2s(lm_ext_texture, avg_ext_texture_samples[:,i]) for i in range(len(rs))]).T

        scores = contrast_map[sample_ys, sample_xs] + np.exp(-np.minimum(int_chi2s, ext_chi2s)/.1)

    #     scores = contrast_map[sample_ys, sample_xs]

        best_sample = scores.argmax(axis=1)
        snapped_pts = np.c_[sample_ys[range(n), best_sample], sample_xs[range(n), best_sample]]
        
        pts = snapped_pts

    return pts, scores.max(axis=1).sum()

In [39]:
def rigid_transform_from_pairs(X,Y):
    '''
    X, Y are n-by-2 matrices
    Y = RX
    '''
    
    Xcentroid = X.mean(axis=0)
    Ycentroid = Y.mean(axis=0)
    
    Xcentered = X - Xcentroid
    Ycentered = Y - Ycentroid
    
    U, S, V = np.linalg.svd(np.dot(Xcentered.T, Ycentered))
    M = [[1, 0], [0, np.linalg.det(np.dot(V, U.T))]]
    R = np.dot(V, np.dot(M, U.T))
    angle = np.arctan2(R[1,0], R[0,0])
    t = Ycentroid.T - np.dot(R, Xcentroid.T)
    
    T = np.eye(3)
    T[:2, :2] = R
    T[:2, 2] = t
   
    return T, angle

def rigid_transform_to(pts1, T):
    pts1_trans = np.dot(T, np.column_stack([pts1, np.ones((pts1.shape[0],))]).T).T
    pts1_trans = pts1_trans[:,:2]/pts1_trans[:,-1][:,np.newaxis]
    return pts1_trans


In [109]:
snapped = np.c_[shifted_ys_best, shifted_xs_best]

snapped_history = []

for i in range(100):
    snapped_history.append(snapped)
    snapped_new, _ = snap(snapped, Gmax)
#     print np.sum(np.abs(snapped - snapped_new))
    if np.sum(np.abs(snapped - snapped_new)) < 10:
        break
    snapped = snapped_new

In [103]:
snapped = np.c_[shifted_ys_best, shifted_xs_best]

scores = []
new_pts = []
samples = []

for iter_ind in range(20):
        
    r = np.random.randint(0,len(snapped),2)
    snapped_new_sampled, _ = snap(snapped, Gmax, iterations=10)
    T, a = rigid_transform_from_pairs(snapped[r], snapped_new_sampled[r])
    snapped_new_hypothesis = rigid_transform_to(snapped, T).astype(np.int)
    snapped_new_consensus, score = snap(snapped_new_hypothesis, Gmax, snap_explore_radius=1, iterations=10)

    print iter_ind, score

    scores.append(score)
    new_pts.append(snapped_new_consensus)
    samples.append(r)

0 40.6067404558
1 nan
2 57.7429791071
3 36.8225149958
4 nan
5 60.0589084139
6 68.6065242693
7 64.3426460241
8 nan
9 52.6805961445
10 55.229302551
11 38.8268407179
12 64.8035679239
13 55.586598863
14 55.2922909811
15 nan
16 67.9496971726
17 62.989036161
18 42.1249655512
19 65.8949858045


In [106]:
best_iter = np.argmax(np.nan_to_num(scores))
snapped_new = new_pts[best_iter]
best_sample = samples[best_iter]
# snapped_new = new_pts[0]

In [107]:
# bg = Gmax_viz[:,:,:3].copy()
bg = dm.image_rgb.copy()

overlay = np.zeros_like(dm.image, np.bool)
overlay[snapped[best_sample,0], snapped[best_sample,1]] = 1
overlay = binary_dilation(overlay, disk(5))

bg[overlay] = [0,1,0]

overlay = np.zeros_like(dm.image, np.bool)
overlay[snapped_new_sampled[best_sample,0], snapped_new_sampled[best_sample,1]] = 1
overlay = binary_dilation(overlay, disk(5))

bg[overlay] = [0,0,1]

overlay = np.zeros_like(dm.image, np.bool)
overlay[snapped_new[:,0], snapped_new[:,1]] = 1
overlay = binary_dilation(overlay, disk(5))

bg[overlay] = [1,0,0]

display(bg)

/oasis/projects/nsf/csd395/yuncong/Brain/notebooks/tmp.jpg

In [111]:
# bg = Gmax_viz[:,:,:3].copy()
bg = dm.image_rgb.copy()

overlay = np.zeros_like(dm.image, np.bool)
overlay[snapped_history[0][:,0], snapped_history[0][:,1]] = 1
overlay = binary_dilation(overlay, disk(5))

bg[overlay] = [0,1,0]

# overlay = np.zeros_like(dm.image, np.bool)
# overlay[snapped_history[1][:,0], snapped_history[1][:,1]] = 1
# overlay = binary_dilation(overlay, disk(5))

# bg[overlay] = [0,0,1]

overlay = np.zeros_like(dm.image, np.bool)
overlay[snapped_history[-1][:,0], snapped_history[-1][:,1]] = 1
overlay = binary_dilation(overlay, disk(5))

bg[overlay] = [1,0,0]

display(bg)

/oasis/projects/nsf/csd395/yuncong/Brain/notebooks/tmp.jpg